In [1]:
import cv2
from utils import display_cv2_image_mat
from backend.preprocess import straighten_image
from backend.table import *
os.environ["EASYOCR_MODULE_PATH"] = "backend/EasyOCR"
from backend.EasyOCR.easyocr import Reader
import torch
import sys
import math
# from backend.EasyOCR import reader

In [2]:
image = cv2.imread("sample/medium.png")
img_size = image.size
print(img_size)

1590000


In [3]:
#Clear cache, expect 0, if not 0, restart workbook for accurate result
torch.cuda.empty_cache()
torch.cuda.memory_allocated() # bộ nhớ được phân bổ ban đầu = 0

0

In [4]:
#model  = 307272192 B, sum of JA and EN model
EASYOCR_READER = Reader(['ja','en'], gpu=True)
model_memory = torch.cuda.memory_allocated() # bộ nhớ được phân bổ cho mô hình
print(model_memory)

net 84978176
model 307272192
307272192


In [5]:
# Parameter
decoder = 'greedy'
beamWidth= 5
workers = 0
allowlist = None
blocklist = None
detail = 1,
paragraph = False
min_size = 20
contrast_ths = 0.1
adjust_contrast = 0.5
filter_ths = 0.003,
text_threshold = 0.7
low_text = 0.4
link_threshold = 0.4
canvas_size = 2560
mag_ratio = 1.
slope_ths = 0.1
ycenter_ths = 0.5
height_ths = 0.5
width_ths = 0.5
add_margin = 0.1

In [6]:
from backend.EasyOCR.utils import reformat_input
img, img_cv_grey = reformat_input(image)

In [7]:
horizontal_list, free_list = EASYOCR_READER.detect(img, min_size, text_threshold,\
                                                 low_text, link_threshold,\
                                                 canvas_size, mag_ratio,\
                                                 slope_ths, ycenter_ths,\
                                                 height_ths,width_ths,\
                                                 add_margin, False)
x = torch.cuda.memory_allocated()
print(x)
bound_memory = x - model_memory
print(bound_memory)
# Bounds is stored in ram, and doesn't use GPU memory

test_net 334127616
307272192
0


In [8]:
#number of payload to load into network
payload = len(horizontal_list) + len(free_list)
print(payload) # =len(result)

33


In [9]:
print(horizontal_list)
print(free_list)

[[39, 399, 19, 39], [45, 146, 63, 89], [215, 323, 67, 87], [382, 450, 64, 88], [549, 597, 67, 85], [885, 976, 65, 91], [1050, 1140, 63, 89], [47, 95, 93, 113], [214, 324, 92, 116], [383, 437, 93, 113], [885, 997, 93, 113], [1053, 1141, 93, 113], [89, 159, 125, 145], [345, 407, 125, 145], [767, 823, 125, 145], [927, 999, 125, 145], [1111, 1149, 125, 145], [214, 358, 152, 176], [951, 975, 155, 175], [1055, 1103, 157, 173], [214, 344, 179, 206], [1053, 1103, 181, 201], [217, 405, 209, 229], [1053, 1103, 209, 229], [214, 358, 233, 258], [1053, 1103, 235, 255], [216, 298, 262, 286], [1053, 1103, 263, 283], [214, 306, 289, 315], [1053, 1103, 291, 311], [555, 601, 345, 365], [951, 973, 347, 365], [1055, 1103, 349, 365]]
[]


In [10]:
batch_size =  8
loop = math.ceil(payload/batch_size)
print("number of expected loops:" ,loop) 
result = EASYOCR_READER.recognize(img_cv_grey, horizontal_list, free_list,\
                                decoder, beamWidth, batch_size,\
                                workers, allowlist, blocklist, detail,\
                                paragraph, contrast_ths, adjust_contrast,\
                                filter_ths, False)
# print(result)
# sum_tensor_size đặt ngoài vòng for, được in ra 2 lần => chạy qua hàm recognizer_predict ứng với mỗi lần forward 2 lần dù loop = 1 <= 'predict two round'
#309639680

number of expected loops: 5
1
length_for_pred  32
text_for_pred  7424
model_size 309639680
image_tensors_size 2359296
model_output_size 28770528
2
length_for_pred  32
text_for_pred  7424
model_size 367230464
image_tensors_size 2359296
model_output_size 28770528
3
length_for_pred  32
text_for_pred  7424
model_size 367230464
image_tensors_size 2359296
model_output_size 28770528
4
length_for_pred  32
text_for_pred  7424
model_size 367230464
image_tensors_size 2359296
model_output_size 28770528
5
length_for_pred  4
text_for_pred  928
model_size 365159424
image_tensors_size 294912
model_output_size 3596316
sum_prob 118678428
sum_tensor_size 9732096
1
length_for_pred  8
text_for_pred  1856
model_size 307864576
image_tensors_size 589824
model_output_size 7192632
sum_prob 7192632
sum_tensor_size 589824


In [11]:
max_VGA_need = 367230464
model_size = 307272192
image_tensor_size = 2359296
model_output_size = 28770528

In [12]:
(model_size + image_tensor_size + model_output_size * 2) - max_VGA_need

-57920

In [13]:
sum_model_output = 118678428+7192632
print(sum_model_output)

125871060


In [14]:
sum_tensors_size=9732096+589824
print(sum_tensors_size)

10321920


In [15]:
sum_tensors_size/img_size

6.49177358490566

In [16]:
model_output_size/image_tensor_size

12.194539388020834

In [17]:
"""
    image_tensor_size = img_size * 6 / loop
    model_output_size = image_tensor_size * 12 = img_size * 72 / loop
"""

'\n    image_tensor_size = img_size * 6 / loop\n    model_output_size = image_tensor_size * 12 = img_size * 72 / loop\n'

In [18]:
"""
    max_VGA_need    = model_size + image_tensor_size + model_output_size*2
                    = 307272192 + (img_size*6 + img_size*72*2)/loop
                    = 307272192 + img_size*150/loop
                    = 307272192 + img_size*150/(payload/batch_size)
                    = 307272192 + img_size*150*batch_size/payload
"""

'\n    max_VGA_need    = model_size + image_tensor_size + model_output_size*2\n                    = 307272192 + (img_size*6 + img_size*72*2)/loop\n                    = 307272192 + img_size*150/loop\n                    = 307272192 + img_size*150/(payload/batch_size)\n                    = 307272192 + img_size*150*batch_size/payload\n'

In [19]:
(307272192+1590000*150*4/15)-367230464

3641728.0

In [20]:
!nvidia-smi

Fri Dec 18 10:42:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0    N/A /  N/A |   1697MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [21]:
GPU_memory = 2048*1048576
print(GPU_memory)

2147483648


In [22]:
torch.cuda.get_device_properties(0).total_memory

2147483648

In [23]:
max_VGA_need = 322265088

In [24]:
loop = GPU_memory/max_VGA_need
print(loop)

6.663717938941


In [25]:
batch_size = 307272192*payload/(GPU_memory-150*img_size)
print(batch_size)

5.311717754430969


In [26]:
del EASYOCR_READER